In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [817 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
# import reviews
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [4]:
# Filter DataFrame total_votes count is equal to or greater than 20
total_votes_df = df.filter(df.total_votes >= 20)
total_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   13446431|R1QXNQBTH7LIUB|B00V03PK74|     159528888|Kalamera freestan...|Major Appliances|          3|            9|         22|   N|                Y|They claim that i...|They claim that i...| 2015-08-31|
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|    

In [5]:
# Filter DataFrame helpful_votes is equal to or greater than 50%
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|           83|         89|   N|                Y|Warning! Long Rev...|Where do I begin?...| 2015-08-31|
|         US|   13254956| R5XKK92G5N9FU|B00EM2JPLW|     824392206|Samsung NE58F9500...|Major Appliances|          1|    

In [6]:
# Create Vine paid DataFrame
vine_paid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
vine_paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   31302915|R20O97P5V9CZ21|B00P7QI4IM|     901651668|Whynter BR-130SB ...|Major Appliances|          3|           21|         27|   Y|                N|Nice but had some...|I was very excite...| 2015-06-07|
|         US|   38320678| RCNB7YGLWKCV5|B00NQFSSWS|     692404913|Samsung Counter T...|Major Appliances|          5|    

In [7]:
# Create Vine unpaid DataFrame
vine_unpaid_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
vine_unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   36792525|R3BC75VKJK6LNV|B00RL014P2|       6980707|The Laundry Alter...|Major Appliances|          4|           83|         89|   N|                Y|Warning! Long Rev...|Where do I begin?...| 2015-08-31|
|         US|   13254956| R5XKK92G5N9FU|B00EM2JPLW|     824392206|Samsung NE58F9500...|Major Appliances|          1|    

In [8]:
# total paid reviews
total_paid_reviews = vine_paid_df.count()
total_paid_reviews

35

In [9]:
# total paid 5-star reviews
paid_five_star_reviews = vine_paid_df.filter(vine_paid_df.star_rating == 5).count()
paid_five_star_reviews

18

In [10]:
# paid percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

51.42857142857142

In [11]:
# total unpaid reviews
total_unpaid_reviews = vine_unpaid_df.count()
total_unpaid_reviews

4957

In [12]:
# total unpaid 5-star reviews
unpaid_five_star_reviews = vine_unpaid_df.filter(vine_unpaid_df.star_rating == 5).count()
unpaid_five_star_reviews

1963

In [13]:
# unpaid percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

39.60056485777688

In [14]:
# double check using pyspark
from pyspark.sql.functions import col,when,count,lit
ratings_total_df = helpful_votes_df.groupBy("Vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("Percentage_Total")).show()

+----+-------------+--------------------+-----------------+
|Vine|Total_Reviews|Total_5_Star_Reviews| Percentage_Total|
+----+-------------+--------------------+-----------------+
|   Y|           35|                  18|51.42857142857142|
|   N|         4957|                1963|39.60056485777688|
+----+-------------+--------------------+-----------------+

